In [ ]:
WORKDIR = "/mnt/scratch/daniel/Data/AnaContML"

import sys
sys.path.append(WORKDIR+"/src")
import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"

import torch
from torch.utils.data import DataLoader
import load_data
import datetime
from pytorch_lightning.loggers import TensorBoardLogger
import pytorch_lightning as pl
from pytorch_lightning.callbacks import EarlyStopping, ModelCheckpoint
from pytorch_lightning.plugins.environments import LightningEnvironment
import json
import numpy as np
import h5py
import time
import matplotlib.pyplot as plt
# from pympler import asizeof
from scipy.optimize import curve_fit

In [ ]:
def create_datasets(config):
    data = np.load(config["PATH_TRAIN"])
    train, validation = torch.utils.data.random_split(data, [int(data.__len__()*config["SPLIT"]), int(data.__len__())-int(data.__len__()*config["SPLIT"])], generator=torch.Generator().manual_seed(42))
    return train, validation


data_dict = {}
def validate():
    ## JSON File contains full information about entire run (model, data, hyperparameters)
    MODEL_NAME = "GNN_1_base"
    config = json.load(open(WORKDIR+'/_runs/confmod_graph_neural_network_MIT_w100_n100_skew1.json'))[MODEL_NAME]
    print(config)
    config["batch_size"] = 1

    # ''' Dataloading '''
    train_data, validation_data = create_datasets(config)
    train_data = np.array(train_data)
    validation_data = np.array(validation_data)

    # ### > Single HDF5 file containing training and validation data 
    ld = __import__("load_data", fromlist=['object'])
    train_set = getattr(ld, config["DATA_LOADER"])(config, train_data)
    validation_set = getattr(ld, config["DATA_LOADER"])(config, validation_data)
    train_dataloader = DataLoader(train_set, batch_size=config["batch_size"], shuffle=False)
    validation_dataloader = DataLoader(validation_set, batch_size=config["batch_size"], shuffle=False)

    ''' Storage '''
    t_samples = np.asarray(train_set).shape[0]
    v_samples = np.asarray(validation_set).shape[0]

    data_dict["train"] = {}
    data_dict["train"]["green"] = torch.zeros((t_samples, config["tau_steps"]))
    data_dict["train"]["prediction"] = torch.zeros((t_samples, config["omega_steps"]))
    data_dict["train"]["target"] = torch.zeros((t_samples, config["omega_steps"]))
    data_dict["train"]["p_coefficients"] = torch.zeros((t_samples, config["n_nodes"]))
    data_dict["train"]["vectors"] = torch.zeros((t_samples, config["n_nodes"], config["omega_steps"]))

    data_dict["valid"] = {}
    data_dict["valid"]["green"] = torch.zeros((v_samples, config["tau_steps"]))
    data_dict["valid"]["prediction"] = torch.zeros((v_samples, config["omega_steps"]))
    data_dict["valid"]["target"] = torch.zeros((v_samples, config["omega_steps"]))
    data_dict["valid"]["p_coefficients"] = torch.zeros((v_samples, config["n_nodes"]))
    data_dict["valid"]["vectors"] = torch.zeros((v_samples, config["n_nodes"], config["omega_steps"]))

    beta = [10, 20, 30, 40]
    data_dict["ctqmc"] = {}
    for b in beta:
        data_dict["ctqmc"][b] = {}
        data_dict["ctqmc"][b]["green"] = torch.zeros((40, config["tau_steps"]))
        data_dict["ctqmc"][b]["prediction"] = torch.zeros((40, config["omega_steps"]))
        data_dict["ctqmc"][b]["p_coefficients"] = torch.zeros((40, config["n_nodes"]))
        data_dict["ctqmc"][b]["vectors"] = torch.zeros((40, config["n_nodes"], config["omega_steps"]))
        data_dict["ctqmc"][b]["U"] = torch.zeros(40)

    
    ''' Model setup '''
    wrapers = __import__("wrappers.wrapers", fromlist=['object'])#.wrapers
    model = getattr(wrapers, config["MODEL_WRAPER"])(config)

    ''' Model loading from save file '''
    if config["continue"] == True:
        SAVEPATH = config["SAVEPATH"]
        if torch.cuda.is_available():
            checkpoint = torch.load(SAVEPATH)
        else:
            checkpoint = torch.load(SAVEPATH, map_location=torch.device('cpu'))
        model.load_state_dict(checkpoint['state_dict'])
        print(" >>> Loaded checkpoint")
    
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        model = model.to(device)
        # model.eval()
    

    start = time.time()
    with torch.no_grad():
        
        for b in beta:
            PATH = WORKDIR+f"/valid_scripts/gtau_beta{str(b)}_w2dyn/beta{b}/"
            gtau_ctqmc = []
            nono = []
            _ = np.load(PATH+"data.npz", allow_pickle=True)
            data = {key: _[key].item() for key in _}
            for U in data.keys():
                gtau_ctqmc.append(data[U]["gtau"][0,0,:])
                nono.append([np.zeros_like(gtau_ctqmc) + int(U)][0][0])
            gtau_ctqmc = np.array(gtau_ctqmc)
            nono = np.array(nono)            
    
            step = 10
            gtau_ctqmc = np.concatenate((
                gtau_ctqmc[:, 0][:, None],                     # first point
                gtau_ctqmc[:, step:-step:step],               # 98 intermediate points
                gtau_ctqmc[:, -1][:, None]                    # last point
            ), axis=1)
            nono = np.concatenate((
                nono[:, 0][:, None],                     # first point
                nono[:, step:-step:step],               # 98 intermediate points
                nono[:, -1][:, None]                    # last point
            ), axis=1)
            data_ctqmc = np.concatenate((gtau_ctqmc[:,None,:], nono[:,None,:]), axis=1)
            
            ctqmc_set = getattr(ld, config["DATA_LOADER"])(config, data_ctqmc)
            ctqmc_dataloader = DataLoader(ctqmc_set, batch_size=config["batch_size"], shuffle=False)
    
            for n, batch in enumerate(ctqmc_dataloader):
                # Move batch data to GPU
                batch_gpu = {}
                for k, v in batch.items():
                    if torch.is_tensor(v):
                        batch_gpu[k] = v.to(device, non_blocking=True)
                    else:
                        batch_gpu[k] = v
    
                out = model.model.forward(batch_gpu)
                data_dict["ctqmc"][b]["green"][n] = batch_gpu["node_feature"][0,0,config["omega_steps"]:]
                data_dict["ctqmc"][b]["prediction"][n] = out[0][0]
                data_dict["ctqmc"][b]["vectors"][n] = out[1][0]
                data_dict["ctqmc"][b]["p_coefficients"][n] = out[2][0,:,0]
                data_dict["ctqmc"][b]["U"][n] = batch_gpu["target"][0][0]
        print("  CTQMC DONE   ")
            
        ## TRAINING DATA
        for n, batch in enumerate(train_dataloader):
            # Move batch data to GPU
            batch_gpu = {}
            for k, v in batch.items():
                if torch.is_tensor(v):
                    batch_gpu[k] = v.to(device, non_blocking=True)
                else:
                    batch_gpu[k] = v

            out = model.model.forward(batch_gpu)            
            data_dict["train"]["green"][n] = batch_gpu["node_feature"][0,0,config["omega_steps"]:]
            data_dict["train"]["target"][n] = batch_gpu["target"]
            # data_dict["train"]["prediction"][n] = model.model.forward(batch_gpu)
            data_dict["train"]["prediction"][n] = out[0][0]
            data_dict["train"]["vectors"][n] = out[1][0]
            data_dict["train"]["p_coefficients"][n] = out[2][0,:,0]
            
            
            call_time = 1e4
            if n % call_time == 0:
                end = time.time()
                print(f"{n} / {len(train_dataloader)}    Step time: {end - start:.4f} sec")
                remaining = (len(train_dataloader) - n) * (end - start) / 60 / call_time
                print(f"Estimated time left: {remaining:.2f} minutes")
                start = time.time()

        ### VALIDATION DATA
        for n, batch in enumerate(validation_dataloader):
            # Move batch data to GPU
            batch_gpu = {}
            for k, v in batch.items():
                if torch.is_tensor(v):
                    batch_gpu[k] = v.to(device, non_blocking=True)
                else:
                    batch_gpu[k] = v

            out = model.model.forward(batch_gpu)
            data_dict["valid"]["green"][n] = batch_gpu["node_feature"][0,0,config["omega_steps"]:]
            data_dict["valid"]["target"][n] = batch_gpu["target"]
            # data_dict["valid"]["prediction"][n] = model.model.forward(batch_gpu)
            data_dict["valid"]["prediction"][n] = out[0][0]
            data_dict["valid"]["vectors"][n] = out[1][0]
            data_dict["valid"]["p_coefficients"][n] = out[2][0,:,0]
            
            call_time = 5e3
            if n % call_time == 0:
                end = time.time()
                print(f"{n} / {len(validation_dataloader)}    Step time: {end - start:.4f} sec")
                remaining = (len(validation_dataloader) - n) * (end - start) / 60 / call_time
                print(f"Estimated time left: {remaining:.2f} minutes")
                start = time.time()




def main():
    validate()

if __name__ == '__main__':
    main()



In [ ]:
def save_dict_to_hdf5_group(h5file, path, dictionary):
    """
    Recursively save a nested dictionary to HDF5 groups
    """
    for key, item in dictionary.items():
        key_str = str(key)
        if isinstance(item, dict):
            grp = h5file.create_group(f"{path}/{key_str}")
            save_dict_to_hdf5_group(h5file, f"{path}/{key_str}", item)
        else:
            # assume item is array-like or scalar
            h5file.create_dataset(f"{path}/{key_str}", data=np.array(item))
            
with h5py.File("GNN_1_base_MIT_skew_2025-11-28_100_bs100_v2.h5", "w") as f:
    save_dict_to_hdf5_group(f, "train", data_dict["train"])
    save_dict_to_hdf5_group(f, "valid", data_dict["valid"])
    save_dict_to_hdf5_group(f, "ctqmc", data_dict["ctqmc"])